In [1]:
! pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from google.colab import drive
drive.mount('/content/drive')

! cp /content/drive/MyDrive/kaggle.json ~/
! mkdir ~/.kaggle
! mv ~/kaggle.json ~/.kaggle/kaggle.json
! chmod 600 ~/.kaggle/kaggle.json

Mounted at /content/drive


In [3]:
! kaggle datasets download -d eldarsultanow/ml-bsd-dataset-integers

 98% 269M/274M [00:03<00:00, 112MB/s] 
100% 274M/274M [00:03<00:00, 72.8MB/s]


In [4]:
! unzip ml-bsd-dataset-integers.zip

Archive:  ml-bsd-dataset-integers.zip
  inflating: rank-vs-ap-integers.csv  


In [5]:
import pandas as pd
import numpy as np

In [6]:
df = pd.read_csv("rank-vs-ap-integers.csv", index_col=False, header=None)
df.drop_duplicates(inplace=True)
df.dropna(inplace=True)

In [7]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,1222,1223,1224,1225,1226,1227,1228,1229,1230,1231
0,1,7073163673,-1,1,1,2,-1,3,-2,0,...,-72,85,172,158,-126,-138,44,-85,85,-13
1,1,7578874153,-1,1,0,0,-2,6,-4,0,...,-119,-97,-170,-31,-169,-76,-37,42,-78,122
2,1,7526030717,-1,0,2,0,-1,0,2,3,...,172,-186,93,-168,66,120,-74,-78,-96,100
3,1,7078096709,-1,1,0,-3,2,-4,-6,-1,...,103,127,-97,-24,122,-67,67,29,-95,56
4,0,7886103067,1,-1,0,0,5,-1,2,-4,...,138,-29,-34,9,-83,64,140,102,62,139


In [8]:
df[0].value_counts()

1    62710
0    42282
2    30000
3    13616
4     1335
5       48
6        1
Name: 0, dtype: int64

In [9]:
df_3 = df.loc[df[0] == 3].sample(n=1335, random_state=1)
df_4 = df.loc[df[0] == 4]
df_3_4 = pd.concat([df_3, df_4])

df_1 = df.loc[df[0] == 1].sample(n=42282, random_state=1)
df_0 = df.loc[df[0] == 0]
df_0_1 = pd.concat([df_0, df_1])

df_sub = df_3_4
df_sub[0].value_counts()

3    1335
4    1335
Name: 0, dtype: int64

In [ ]:
#converts a 2d integer matrix into a 1d bool matrix
def binary_mask_2d(ap_values):
    diffs = np.abs(ap_values[:, 1:] - ap_values[:, :-1])
    sums = np.sum(diffs, axis=1)

    median_sum = np.median(sums)
    std_sum = np.std(sums)

    binary_mask = np.zeros((len(ap_values),))
    for i in range(len(ap_values)):
        if sums[i] > median_sum + std_sum:
            binary_mask[i] = 1
    return binary_mask

In [52]:
ap_cols = range(2, 10)
ap_col_mat = df_sub[ap_cols].to_numpy()

X = df_sub[ap_cols].values
#X = np.sign(ap_col_mat)

y = df_sub[0]
#y = df_sub[0].astype('category').cat.codes.values

print(X.shape)
print(y.shape)

(2670, 8)
(2670,)


In [53]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, f1_score

In [58]:
x_var_2d = df_sub.loc[:, ap_cols]
y_var = df_sub.loc[:, 0]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.7, random_state = 10)

In [59]:
print(y_train)

137999    3
144826    3
146372    4
136461    4
140735    4
         ..
142591    4
138994    3
135083    4
145872    3
139465    3
Name: 0, Length: 801, dtype: int64


In [60]:
import lightgbm as lgb

clf = lgb.LGBMClassifier()
clf.fit(X_train, y_train)
y_pred=clf.predict(X_test)

In [61]:
print("RMSE: {:.4f}".format(np.sqrt(mean_squared_error(y_test, y_pred))))
print("R^2 Score: {:.4f}".format(r2_score(y_test, y_pred)))
print("ACC Score: {:.4f}".format(accuracy_score(y_test, y_pred)))

RMSE: 0.0000
R^2 Score: 1.0000
ACC Score: 1.0000


In [64]:
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv1D, MaxPooling1D, Flatten

In [62]:
X_train, X_test, y_train, y_test = train_test_split(x_var_2d, y_var, test_size=0.2, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=1) # 0.25 x 0.8 = 0.2

In [65]:
callback = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)
input_dim = X_train.shape[1]
dropout = 0.2
classifier = Sequential()
classifier.add(Dropout(dropout, input_shape=(input_dim,)))
classifier.add(Dense(units = round(.8*input_dim), activation = 'relu', input_dim = input_dim))
classifier.add(Dropout(.8*dropout))
classifier.add(Dense(units = 1,  activation = 'sigmoid'))
classifier.compile(optimizer = 'Adamax', loss = 'binary_crossentropy', metrics = ['binary_accuracy'])
progress = classifier.fit(X_train, y_train, batch_size = 32, epochs = 60, callbacks = [callback], validation_data=(X_val, y_val), verbose=0)
results = classifier.evaluate(X_test, y_test, batch_size=16)

34/34 [==============================] - 0s 2ms/step - loss: -261.1505 - binary_accuracy: 0.0000e+00


In [ ]:
!pip install sympy

In [ ]:
from sympy import primepi

def get_r_sequence(N:int):
  pi_N = primepi(N)
  res = [None]*pi_N

  for i in range(1,pi_N+1):
    res[i-1] = -1 + i*2/pi_N
  return res

print(len(get_r_sequence(10**2)))
print(get_r_sequence(10**2))